# Stage 04 - Data Acquisition and Ingestion
## Lecture 04
API Pull, Scraping, dll

In [40]:
import os, json, time, datetime as dt, csv, pathlib
from typing import Dict, List
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

#pathlib = library utk pilih direktori folder
DATA_RAW = pathlib.Path("data/raw")

#mkdir untuk bikin folder
DATA_RAW.mkdir(parents=True, exist_ok= True)

load_dotenv()
ALPHA_KEY = os.getenv("ALPHAVANTAGE_API_KEY")
print("Loaded APLHAVANTAGE_API_KEY?", bool(ALPHA_KEY))

#print isi API nya
print(ALPHA_KEY)

Loaded APLHAVANTAGE_API_KEY? True
your_real_api_key_here


In [38]:
from dotenv import load_dotenv
import os

# 1. Load .env explicitly (you can remove dotenv_path if .env is in same folder)
load_dotenv(dotenv_path=".env")

# 2. See what files are in the current working directory
print("Current working dir:", os.getcwd())
print("Files here:", os.listdir())

# 3. Try to read the key
print("ALPHAVANTAGE_API_KEY:", os.getenv("ALPHAVANTAGE_API_KEY"))

Current working dir: /Users/azfina/Documents/bootcamp_Azfina_Anindita/homework/Lecture04
Files here: ['.DS_Store', '.env.txt', '.env', '.ipynb_checkpoints', 'Lecture04.ipynb', 'data']
ALPHAVANTAGE_API_KEY: your_real_api_key_here


# Helper Functions: Validation & Filenames

In [61]:
def safe_stamp():
    return dt.datetime.now().strftime("%Y%m%d-%H%M%S")

In [62]:
safe_stamp()

'20250815-101147'

In [63]:
dt.datetime.now().strftime("%y-%m-%d")

'25-08-15'

In [64]:
def save_filename(prefix: str, meta:Dict [str, str]) -> str:
    mid = "_".join([f"{k}--{str(v).replace(' ', '_')[:20]}" for k, v, in meta.items()])
    return f"{prefix}_{mid}_{safe_stamp()}.csv"

In [65]:
save_filename ("wiki",{"param1":1,"param2":2})

'wiki_param1--1_param2--2_20250815-101149.csv'

In [107]:

def validate_df(df: pd.DataFrame, required_cols: List[str], dtypes_map: Dict[str, str]) -> Dict[str, str]:
    msgs = {}
    
    # Check for missing columns
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        msgs['missing_cols'] = f"Missing columns: {missing}"
    
    # Check column data types
    for col, dtype in dtypes_map.items():
        if col in df.columns:
            try:
                if dtype == 'datetime64[ns]':
                    pd.to_datetime(df[col])
                elif dtype == 'float':
                    pd.to_numeric(df[col])
            except Exception as e:
                msgs[f'dtype_{col}'] = f"Failed to coerce {col} to {dtype}: {e}"
    
    # Count total NA values
    na_counts = df.isna().sum().sum()
    msgs['na_total'] = f"Total NA values: {na_counts}"
    
    return msgs




In [81]:
numbered_list=[]
for idx, i in enumerate(["temperature","humidity"]):
    numbered_list += [str(idx+1)+". "+str(i)]

numbered_list

#[str(idx+1)+". "+str(i) for idx, i in enumerate (["temperature","humidity"])

[i[:2] for i in ["temperature","humidity"]]

['te', 'hu']

In [84]:
def test (x:str):
    return x

test({"1":1})

{'1': 1}

In [94]:
5 if 1 in [1,2] else 4
#kalau 1 ada di 1,2 maka return 5. tapi kalau gak return 4



5

In [95]:
None if 1 not in [1,2] else 4
#kalau 1 GAKada di 1,2 maka return None. tapi kalau gak return 4

4

In [99]:
x = 5

#didalam range [1,2,3,4] kalau dia 1 maka TRUE tapi kalau bukan FALSE
[True if c == 1 else False 
 for c in [1,2,3,4]]

[True, False, False, False]

In [102]:
[None if c not in [1,2] else c 
for c in [1,2,3,4]]

#coba cek isi range [1,2,3,4] kalau bukan [1,2] maka return none; tapi kalau [1,2] maka return c (dirinya sendiri)

[1, 2, None, None]

In [117]:
ALPHA_KEY="D3Q1QKM8OUW9K278"
ALPHA_KEY

'D3Q1QKM8OUW9K278'

In [120]:

SYMBOL = "AAPL"

use_alpha = bool(ALPHA_KEY)
print("Using Alpha Vantage:", use_alpha)

if use_alpha:
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": SYMBOL,
        "outputsize": "compact",
        "apikey": ALPHA_KEY,
        "datatype": "json"
    }
    
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    
    key = [k for k in js.keys() if "Time Series" in k]
    assert key, f"Unexpected response keys: {list(js.keys())}"
    
    series = js[key[0]]
    df_api = (pd.DataFrame(series).T
              .rename_axis('date')
              .reset_index())
    
    # keep a couple columns and coerce types
    df_api = df_api[['date', '5. adjusted close']].rename(columns={'5. adjusted close': 'adj_close'})
    df_api['date'] = pd.to_datetime(df_api['date'])
    df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    print("Alpha Vantage API key not found or empty.")


Using Alpha Vantage: True


AssertionError: Unexpected response keys: ['Information']

In [121]:
print(js)


{'Information': 'Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints'}


## Beautiful Soup

In [124]:
SCRAPE_URL = "https://example.com/market-table"  # replace with permitted page
headers = {"User-Agent": "AFE-Course-Notebook/1.0 (contact: instructor@example.edu)"}

try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find('table')
    rows = []
    for tr in table.find_all('tr'):
        cells = [td.get_text(strip=True) for td in tr.find_all(['td', 'th'])]
        if cells:
            rows.append(cells)
    # assume first row is header
    header, *data = rows
    df_scrape = pd.DataFrame(data, columns=header)

except Exception as e:
    print("Scrape failed (demoing with inline HTML).", e)
    html = """
    <table>
        <tr><th>Ticker</th><th>Price</th></tr>
        <tr><td>AAA</td><td>101.2</td></tr>
        <tr><td>BBB</td><td>98.7</td></tr>
    </table>
    """
    soup = BeautifulSoup(html, 'html.parser')
    rows = []
    for tr in soup.find_all('tr'):
        cells = [td.get_text(strip=True) for td in tr.find_all(['td', 'th'])]
        if cells:
            rows.append(cells)
    header, *data = rows
    df_scrape = pd.DataFrame(data, columns=header)



Scrape failed (demoing with inline HTML). 404 Client Error: Not Found for url: https://example.com/market-table


In [125]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Example URL (replace with a real page you are allowed to scrape)
url = "https://www.worldometers.info/world-population/population-by-country/"

# Send HTTP request
response = requests.get(url)
response.raise_for_status()  # will raise an error if request fails

# Parse HTML content
soup = BeautifulSoup(response.text, "html.parser")

# Find the first table on the page
table = soup.find("table")

# Extract rows
rows = []
for tr in table.find_all("tr"):
    cells = [td.get_text(strip=True) for td in tr.find_all(["td", "th"])]
    if cells:
        rows.append(cells)

# Assume first row is header
header, *data = rows

# Create DataFrame
df = pd.DataFrame(data, columns=header)

print(df.head())  # show first 5 rows


   # Country (or dependency) Population 2025 Yearly Change    Net Change  \
0  1                   India   1,463,865,525         0.89%    12,929,734   
1  2                   China   1,416,096,094      â0.23%  â3,225,184   
2  3           United States     347,275,807         0.54%     1,849,236   
3  4               Indonesia     285,721,236         0.79%     2,233,305   
4  5                Pakistan     255,219,554         1.57%     3,950,390   

  Density (P/KmÂ²) Land Area (KmÂ²) Migrants (net) Fert. Rate Median Age  \
0              492        2,973,190     â495,753       1.94       28.8   
1              151        9,388,211     â268,126       1.02       40.1   
2               38        9,147,420      1,230,663       1.62       38.5   
3              158        1,811,570      â39,509        2.1       30.4   
4              331          770,880   â1,235,336        3.5       20.6   

  Urban Pop % World Share  
0       37.1%      17.78%  
1       67.5%      17.20%  
2 

In [134]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Yahoo Finance Most Active Stocks page
url = "https://finance.yahoo.com/most-active"

# Request the page
headers = {"User-Agent": "Mozilla/5.0"}  # Pretend to be a browser
response = requests.get(url, headers=headers)
response.raise_for_status()

# Parse with BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

# Find the table
table = soup.find("table")  # Yahoo's first table on the page

# Extract data
rows = []
for tr in table.find_all("tr"):
    cells = [td.get_text(strip=True) for td in tr.find_all(["td", "th"])]
    if cells:
        rows.append(cells)

# Assume first row is header
header, *data = rows

# Create DataFrame
df = pd.DataFrame(data, columns=header)

print(df.head())


  Symbol                        Name                     Price   Change  \
0   OPEN  Opendoor Technologies Inc.    3.4300+0.3900(+12.83%)  +0.3900   
1   INTC           Intel Corporation        24.55+0.68(+2.87%)    +0.68   
2     NU            Nu Holdings Ltd.       13.28+1.27(+10.62%)    +1.27   
3   NVDA          NVIDIA Corporation       179.15-2.87(-1.57%)    -2.87   
4   WULF               TeraWulf Inc.         8.84+0.13(+1.49%)    +0.13   

  Change %    Volume Avg Vol (3M) Market Cap P/E Ratio(TTM) 52 WkChange %  \
0  +12.83%  236.331M     194.061M     2.524B             --       +69.83%   
1   +2.87%  144.636M      87.982M   107.433B             --       +14.33%   
2  +10.62%   71.496M      61.716M    64.092B          30.19       -14.76%   
3   -1.57%   68.334M     183.388M     4.369T          57.98       +46.11%   
4   +1.49%   52.207M      50.714M     3.465B             --      +116.13%   

  52 Wk Range  
0              
1              
2              
3              
4     

In [127]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# --- SETTINGS ---
symbol = "AAPL"  # Example: Apple
ALPHA_KEY = "YOUR_ALPHA_VANTAGE_API_KEY"

# --- TRY ALPHA VANTAGE FIRST ---
try:
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={ALPHA_KEY}"
    r = requests.get(url, timeout=20)
    r.raise_for_status()
    js = r.json()

    key = [k for k in js.keys() if "Time Series" in k]
    assert key, f"Unexpected response keys: {list(js.keys())}"

    series = js[key[0]]
    df_api = (
        pd.DataFrame(series).T
        .rename_axis("date")
        .reset_index()
        .rename(columns={
            "1. open": "Open",
            "2. high": "High",
            "3. low": "Low",
            "4. close": "Close",
            "5. volume": "Volume"
        })
    )

    print("✅ Data from Alpha Vantage API")
    print(df_api.head())

except Exception as e:
    print("⚠ API failed, falling back to Yahoo Finance scrape:", e)

    # --- FALLBACK: SCRAPE YAHOO FINANCE ---
    scrape_url = f"https://finance.yahoo.com/quote/{symbol}/history"
    headers = {"User-Agent": "Mozilla/5.0"}
    
    resp = requests.get(scrape_url, headers=headers, timeout=20)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    table = soup.find("table")
    
    rows = []
    for tr in table.find_all("tr"):
        cells = [td.get_text(strip=True) for td in tr.find_all(["td", "th"])]
        if cells:
            rows.append(cells)
    
    header, *data = rows
    df_scrape = pd.DataFrame(data, columns=header)

    print("✅ Data from Yahoo Finance scraping")
    print(df_scrape.head())


✅ Data from Alpha Vantage API
         date      Open      High       Low     Close     Volume
0  2025-08-14  234.0550  235.1200  230.8500  232.7800   51916275
1  2025-08-13  231.0700  235.0000  230.4300  233.3300   69878546
2  2025-08-12  228.0050  230.8000  227.0700  229.6500   55672301
3  2025-08-11  227.9200  229.5600  224.7600  227.1800   61806132
4  2025-08-08  220.8300  231.0000  219.2500  229.3500  113853967


In [139]:
def my_func():
    try:
        print("hello")
    except Exception as e:
        raise e

x = []

try:
    print("bye")
    my_func()
except Exception as e:
    print ("done")

bye
hello
